* https://docs.google.com/document/d/1b7DlLtvrVxihcO65dsBaIjUU-a6aLjdQfNHSOM2PhUM/edit?tab=t.0#heading=h.v0phdvfalpr2

## setup and tryout

In [17]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


```sh
pip install langchain-google-genai langchain langchain_openai beautifulsoup4 langchain-community lxml serpapi google-search-results faiss-cpu langchainhub --upgrade
```

In [18]:
import os

# from openai import OpenAI
import jupyter_black
import tqdm, tqdm.notebook

jupyter_black.load()

assert {"OPENAI_API_KEY", "GOOGLE_API_KEY"} <= set(os.environ)

GEMINI_MODEL_NAME_CLEVER = "gemini-2.0-flash"
GEMINI_MODEL_NAME_FAST = "gemini-2.0-flash-lite"

In [19]:
%%time
from langchain_google_genai import ChatGoogleGenerativeAI

# Make sure to set your GOOGLE_API_KEY environment variable
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

response = llm.invoke("What are the best practices for developing with LangChain?")

print(response.content)

Developing with LangChain effectively requires understanding its core components and adhering to best practices for building robust, reliable, and efficient LLM-powered applications. Here's a breakdown of key areas and recommendations:

**1. Understanding LangChain Core Components**

*   **Models:**  Choose the right LLM (OpenAI, Cohere, Hugging Face, etc.) based on your needs (cost, performance, specific capabilities).  Consider parameters like context window size, token limits, and model pricing.
*   **Prompts:**
    *   **Prompt Templates:**  Use templates to structure prompts consistently, allowing for easy parameterization and reuse.
    *   **Prompt Engineering:**  Experiment with different prompt formulations to optimize performance.  Consider using techniques like few-shot learning (providing example inputs and outputs) and chain-of-thought prompting.
    *   **Prompt Optimization:**  Iteratively refine your prompts based on LLM responses.  Evaluate responses and identify areas

original:

```python
from langchain_openai.chat_models import ChatOpenAI
# chat = ChatOpenAI(openai_api_key="...")
# If you have an envionrment variable set for OPENAI_API_KEY, you can just do:
chat = ChatOpenAI()
chat.invoke("Hello, how are you?") 
```

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Make sure to set your GOOGLE_API_KEY environment variable.
# You can get one from Google AI Studio: https://aistudio.google.com/app/apikey

# Initialize the Gemini chat model
# You can also specify other models like "gemini-1.5-pro"
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
# Invoke the model with a prompt
response = chat.invoke("Hello, how are you?")
print(response.content)

I am doing well, thank you for asking! As a large language model, I don't experience feelings in the same way humans do, but I am functioning correctly and ready to assist you. How can I help you today?


## 85. Chat Models -- Coding

In [ ]:
from IPython.display import Markdown

response = chat.invoke("What is the capital of France?")
Markdown(response.content)

In [ ]:
response.response_metadata

original:

```python
from langchain_core.messages import HumanMessage, SystemMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]
result = chat.invoke(messages)
```

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Ensure your GOOGLE_API_KEY environment variable is set
# 1. Initialize the Gemini chat model
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

# 2. Define the message(s) for the model
text = "What would be a good company name for a company that makes colorful socks?"
messages = [
    SystemMessage(content="You are a helpful assistant that generates company names."),
    HumanMessage(content=text),
]

# 3. Invoke the model with the messages
result = chat.invoke(messages)

# 4. Print the AI's response content
print(result.content)

## 86. Chat Prompt Templates
* https://drive.google.com/file/d/1JoyxZlYfngmXnvrRyo7qqvUoB7qz6il0/view?usp=drive_link

In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate

chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that generates company names"),
        ("human", "{text}"),
    ]
)

result = chat_prompt_template.invoke(
    {
        "text": "What would be a good company name for a company that makes colorful socks?"
    }
)

# model = Cha(model='gpt-4o-mini')

ai_llm_result = chat.invoke(result)
print(ai_llm_result.content)

## 87. Streaming
* https://drive.google.com/file/d/18sGlOZ8AKwON1CXUMnqf9ONfj7bwjSiO/view?usp=drive_link

In [ ]:
import sys
import tqdm, tqdm.notebook

chat = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-lite",
    # streaming=True,
)
for chunk in tqdm.notebook.tqdm(chat.stream("What is the capital of the moon?")):
    print(chunk.content, end="", flush=True)
    sys.stdout.flush()

## 88. Output Parsers
* https://drive.google.com/file/d/1QWwi3AOCHEoMR83zR21sB7zzKdUxVdfO/view?usp=drive_link

In [ ]:
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup to the joke")
    punchline: str = Field(description="The punchline to the joke")


class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="A list of jokes")


parser = PydanticOutputParser(pydantic_object=Joke)

In [ ]:
print(parser.get_format_instructions())

In [ ]:
template = "Answer the user query.\n{format_instructions}\n{query}"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])


messages = chat_prompt.invoke(
    {
        "query": "What is a really funny joke about Python programming?",
        "format_instructions": parser.get_format_instructions(),
    }
)

In [ ]:
chat = ChatOpenAI()
## does not work with Gemini
result = chat.invoke(messages)

In [ ]:
try:
    joke_object = parser.parse(result.content)
    print(joke_object.setup)
    print(joke_object.punchline)
except Exception as e:
    print(e)

In [ ]:
chat = ChatOpenAI(model="gpt-4.1-mini")
structured_llm = chat.with_structured_output(Joke)
result = structured_llm.invoke("What is a really funny joke about Python programming?")

In [ ]:
result

In [ ]:
class Joke(BaseModel):
    setup: str = Field(description="The setup to the joke")
    punchline: str = Field(description="The punchline to the joke")
    explanation: str = Field(
        description="A detailed explanation of why this joke is funny."
    )


class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="A list of jokes")

In [ ]:
chat = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
structured_llm = chat.with_structured_output(Joke)
result = structured_llm.invoke("What is a really funny joke about Python programming?")
result

## 89. Summarizing large amounts of text
* https://colab.research.google.com/drive/11t0e03SThhKRPq9T1M7xg6BcooBFaTkA

### crisp

In [ ]:
# from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain_core.prompts import PromptTemplate

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
docs = loader.load()

llm = ChatGoogleGenerativeAI(
    temperature=0,
    model=GEMINI_MODEL_NAME_CLEVER,
)
chain = load_summarize_chain(llm, chain_type="stuff")

res = chain.invoke(docs)

In [ ]:
res["input_documents"]
Markdown(res["output_text"])

### map reduce

* problem if pages refer to each other (since summaries are done independently)

In [ ]:
from langchain.chains.mapreduce import MapReduceChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import (
    ReduceDocumentsChain,
    MapReduceDocumentsChain,
    StuffDocumentsChain,
)

In [ ]:
llm = ChatGoogleGenerativeAI(temperature=0, model=GEMINI_MODEL_NAME_CLEVER)

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)

# map_chain:
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [ ]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [ ]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=0
)
split_docs = text_splitter.split_documents(docs)

In [ ]:
%%time
# print()
res = map_reduce_chain.invoke(split_docs)

In [ ]:
Markdown(res["output_text"])

### template

In [6]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

# Page 1 --> Page 2 (Refine) --> Page 3 (Refine)

NameError: name 'PromptTemplate' is not defined

In [ ]:
for i, v in enumerate(result["intermediate_steps"]):
    display(Markdown(f"## step {i+1}\n{v}"))

In [ ]:
result["output_text"]

## 91. Document Loaders, Text Splitting, Creating LangChain Documents

https://colab.research.google.com/drive/1YdtBCggWStErmFeP5GBSmEaw04kXeKqD

In [7]:
from bs4 import BeautifulSoup
from langchain_community.document_loaders import TextLoader
import requests

# Get this file and save it locally:
url = "https://github.com/hammer-mt/thumb/blob/master/README.md"

# Save it locally:
r = requests.get(url)

# Extract the text from the HTML:
soup = BeautifulSoup(r.text, "html.parser")
text = soup.get_text()

with open("README.md", "w") as f:
    f.write(text)

loader = TextLoader("README.md")
docs = loader.load()

In [9]:
len(docs)

1

In [10]:
from langchain_core.documents import Document

[Document(page_content="test", metadata={"test": "test"})]

[Document(metadata={'test': 'test'}, page_content='test')]

In [12]:
# Split the text into sentences:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

final_docs = text_splitter.split_documents(loader.load())
len(final_docs)

22

In [15]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain

In [18]:
%%time
chain = load_summarize_chain(llm=chat, chain_type="map_reduce")
res = chain.invoke(
    {
        "input_documents": final_docs,
    }
)

CPU times: user 69.4 ms, sys: 50.5 ms, total: 120 ms
Wall time: 15.3 s


In [20]:
res.keys()
res["output_text"]

'This describes a comprehensive GitHub interface, outlining features for code management, collaboration, security, AI-powered tools (Copilot), automation (Actions), and learning. It includes project-specific views, enterprise-grade security and support options, and links to resources, documentation, and community features. The interface encompasses navigation menus, error messages, and user settings, including a feedback form, saved search management, and appearance customization.'

## 92. Tagging Documents
https://colab.research.google.com/drive/1Gn1IxMqz0RcOaDKVdY7cnzgik0JoQlqZ

In [5]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

In [6]:
from langchain.document_loaders.sitemap import SitemapLoader
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
import pandas as pd

In [7]:
sitemap_loader = SitemapLoader(web_path="https://understandingdata.com/sitemap.xml")
sitemap_loader.requests_per_second = 5
docs = sitemap_loader.load()

Fetching pages: 100%|############################################################################################################################################################| 103/103 [00:03<00:00, 33.36it/s]


In [18]:
# Schema
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "primary_topic": {
            "type": "string",
            "description": "The main topic of the document.",
        },
    },
    "required": ["primary_topic", "sentiment", "aggressiveness"],
}

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
## does not work with Gemini
# llm = ChatGoogleGenerativeAI(temperature=0, model=GEMINI_MODEL_NAME_CLEVER)
chain = create_tagging_chain(schema, llm, output_key="output")

In [19]:
results = []

# Remove the 0:10 to run on all documents:
for index, doc in enumerate(docs[0:10]):
    print(f"Processing doc {index +1}")
    chain_result = chain.invoke({"input": doc.page_content})
    results.append(chain_result["output"])

Processing doc 1
Processing doc 2
Processing doc 3
Processing doc 4
Processing doc 5
Processing doc 6
Processing doc 7
Processing doc 8
Processing doc 9
Processing doc 10


In [21]:
pd.DataFrame(results)

,sentiment,aggressiveness,primary_topic
0,positive,0,AI products
1,positive,3,technology
2,positive,0,Contact
3,positive,2,Software & Data Engineering Services
4,positive,0,Software & Data Engineering
5,neutral,0,Data Engineering
6,positive,0,Data Engineering Services
7,positive,3,React Software Development
8,positive,0,Python software development
9,positive,0,Python software development


### with Pyadntic

In [29]:
# fixes a bug with asyncio and jupyter
import nest_asyncio

nest_asyncio.apply()

from langchain.document_loaders.sitemap import SitemapLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import create_tagging_chain_pydantic
from pydantic import BaseModel, Field
import pandas as pd


# 1. Pydantic Schema Definition
class DocumentTags(BaseModel):
    """Pydantic model for the tags to be extracted from the document."""

    sentiment: str = Field(
        description="The overall sentiment of the document (e.g., positive, negative, neutral)."
    )
    aggressiveness: int = Field(
        description="A rating from 1 to 10 of how aggressive the text is."
    )
    primary_topic: str = Field(description="The main topic of the document.")


# 2. Load Documents
# Note: This can take a moment to run
sitemap_loader = SitemapLoader(web_path="https://understandingdata.com/sitemap.xml")
sitemap_loader.requests_per_second = 5
docs = sitemap_loader.load()

Fetching pages: 100%|############################################################################################################################################################| 103/103 [00:02<00:00, 48.45it/s]


In [35]:
docs[0].metadata

{'source': 'https://understandingdata.com/',
 'loc': 'https://understandingdata.com/',
 'lastmod': '2025-08-17T12:52:13.527Z',
 'changefreq': 'monthly',
 'priority': '1.0'}

In [ ]:
# 3. Initialize Gemini LLM
# Make sure your GOOGLE_API_KEY environment variable is set
# llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-2.0-flash-lite")
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")

# 4. Create the Pydantic Tagging Chain
# This chain is specifically designed to work with Pydantic models
chain = create_tagging_chain_pydantic(DocumentTags, llm)

results = []

# 5. Process Documents
# Using a smaller slice [0:3] for a quick demonstration
for index, doc in tqdm.notebook.tqdm(list(enumerate(docs[:10]))):
    print(f"--- Processing doc {index + 1} ---")

    # The input to invoke is the document content
    chain_result = chain.invoke({"input": doc.page_content})

    # The result is a Pydantic object, which we convert to a dict
    # Access the result via the "function" key
    # tag_data = chain_result["function"].dict()
    tag_data = chain_result["text"]
    results.append(tag_data)

    print(tag_data)

In [34]:
# Optional: Display results in a DataFrame
df = pd.DataFrame(map(dict, results))
print("\n--- Final Results ---")
print(df)


--- Final Results ---
  sentiment  aggressiveness                         primary_topic
0  positive               3           Software & Data Engineering
1  positive               3                            technology
2   neutral               2                   Contact Information
3  positive               3  Software & Data Engineering Services
4  positive               3                  Software Engineering
5   neutral               5                      Data Engineering
6  positive               3             Data Engineering Services
7  positive               3            React Software Development
8  positive               3           Python Software Development
9  positive               3           Python Software Development


## 93. Tracing with LangSmith
* https://colab.research.google.com/drive/1Sf-_1QP92iuJmFkykCufOYRkOB7tkliU
* https://smith.langchain.com
* https://serpapi.com/

In [5]:
assert {"LANGCHAIN_API_KEY", "SERPAPI_API_KEY"} <= set(os.environ)

In [6]:
import uuid

unique_id = uuid.uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY"

# # Used by the agent in this tutorial
# os.environ["OPENAI_API_KEY"] = ""
# os.environ["SERPAPI_API_KEY"] = "SERPAPI_API_KEY"

In [7]:
from langsmith import Client

client = Client()

In [8]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

# llm = ChatOpenAI(temperature=0)
llm = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_CLEVER, temperature=0)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

/var/folders/02/m1j4q6jj79zbjw0qtxcldpnc0000gn/T/ipykernel_3407/2786892264.py:8: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [9]:
import asyncio
import time
import tenacity

inputs = [
    "How many people live in canada as of 2023?",
    "who is dua lipa's boyfriend? what is his age raised to the .43 power?",
    "what is dua lipa's boyfriend age raised to the .43 power?",
    "how far is it from paris to boston in miles",
    "what was the total number of points scored in the 2023 super bowl? what is that number raised to the .23 power?",
    "what was the total number of points scored in the 2023 super bowl raised to the .23 power?",
    "how many more points were scored in the 2023 super bowl than in the 2022 super bowl?",
    "what is 153 raised to .1312 power?",
    "who is kendall jenner's boyfriend? what is his height (in inches) raised to .13 power?",
    "what is 1213 divided by 4345?",
]
results = []


async def arun(agent, input_example):
    try:
        return await agent.arun(input_example)
    except Exception as e:
        # The agent sometimes makes mistakes! These will be captured by the tracing.
        return e


@tenacity.retry(stop=tenacity.stop_after_attempt(4), wait=tenacity.wait_fixed(30))
def run(agent, input_example: str) -> str:
    return agent.invoke(input_example)


# for input_example in inputs:
#     results.append(arun(agent, input_example))
# results = await asyncio.gather(*results)

for input_example in tqdm.notebook.tqdm(inputs):
    time.sleep(2)
    results.append(run(agent, input_example))

  0%|          | 0/10 [00:00<?, ?it/s]

}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 21
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 19
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 15
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 7
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 51
}
].
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 58
}
].
}
, links {
  description: "Learn more ab

In [10]:
from langchain.callbacks.tracers.langchain import wait_for_all_tracers

# Logs are submitted in a background thread to avoid blocking execution.
# For the sake of this tutorial, we want to make sure
# they've been submitted before moving on. This is also
# useful for serverless deployments.
wait_for_all_tracers()

In [11]:
## gemini
import pandas as pd
from IPython.display import HTML

HTML(pd.DataFrame(results).to_html())

,input,output
0,How many people live in canada as of 2023?,40.08 million
1,who is dua lipa's boyfriend? what is his age raised to the .43 power?,"Dua Lipa's boyfriend is Callum Turner. His age, 35, raised to the power of 0.43 is approximately 4.61."
2,what is dua lipa's boyfriend age raised to the .43 power?,4.612636795281377
3,how far is it from paris to boston in miles,The distance from Paris to Boston is approximately 3440 miles.
4,what was the total number of points scored in the 2023 super bowl? what is that number raised to the .23 power?,The total number of points scored in the 2023 Super Bowl was 73. 73 raised to the power of 0.23 is approximately 2.682651500990882.
5,what was the total number of points scored in the 2023 super bowl raised to the .23 power?,2.682651500990882
6,how many more points were scored in the 2023 super bowl than in the 2022 super bowl?,30
7,what is 153 raised to .1312 power?,1.9347796717823205
8,who is kendall jenner's boyfriend? what is his height (in inches) raised to .13 power?,Kendall Jenner's boyfriend is Devin Booker. His height (in inches) raised to the power of 0.13 is approximately 1.76.
9,what is 1213 divided by 4345?,0.2791714614499425


tracing result: https://drive.google.com/drive/folders/1_4hnRpTYZxO_JLBDDE2HCESQW0eHE6Bj

## 94. LangChain Hub
* https://colab.research.google.com/drive/1lxCk4cnk60rzmu0Wz6pzK-sUmWGca5b0
* https://docs.smith.langchain.com/prompt_engineering/how_to_guides#prompt-hub

In [12]:
from langchain import hub

In [13]:
prompt = hub.pull("homanp/question-answer-pair")
prompt_two = hub.pull("gitmaxd/synthetic-training-data")
prompt_three = hub.pull("rlm/text-to-sql")
rag_prompt = hub.pull("rlm/rag-prompt")

In [14]:
prompt

ChatPromptTemplate(input_variables=['context', 'data_format', 'number_of_pairs'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'homanp', 'lc_hub_repo': 'question-answer-pair', 'lc_hub_commit_hash': 'ee1cda5f28d2e6992e2b5a782edaa5711092246ea914671133bd24296a2f63de'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'data_format', 'number_of_pairs'], input_types={}, partial_variables={}, template='You are an AI assistant tasked with generating question and answer pairs for the given context using the given format. Only answer in the format with no other text. You should create the following number of question/answer pairs: {number_of_pairs}. Return the question/answer pairs as a Python List. Each dict in the list should have the full context provided, a relevant question to the context and an answer to the question.\n\nFormat:\n{data_format}\n\nContext:\n{context}\n'), additional_kwargs={})])

In [15]:
prompt_two

PromptTemplate(input_variables=['EXAMPLE', 'NUMBER', 'PERSPECTIVE', 'SEED_CONTENT'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'gitmaxd', 'lc_hub_repo': 'synthetic-training-data', 'lc_hub_commit_hash': '61aef88bc285cb07783a5575f2f8d504a7cc7d1b2e71fdb63d91dcedfefceb5c'}, template='Utilize Natural Language Processing techniques and Generative AI to create new Question/Answer pair textual training data for OpenAI LLMs by drawing inspiration from the given seed content: {SEED_CONTENT} \n\nHere are the steps to follow:\n\n1. Examine the provided seed content to identify significant and important topics, entities, relationships, and themes. You should use each important topic, entity, relationship, and theme you recognize. You can employ methods such as named entity recognition, content summarization, keyword/keyphrase extraction, and semantic analysis to comprehend the content deeply.\n\n2. Based on the analysis conducted in the first step, employ a generative language

In [16]:
rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [17]:
print(rag_prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [18]:
# Load docs
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Store splits
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

# RAG prompt
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

# LLM
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vectorstore.as_retriever(), chain_type_kwargs={"prompt": prompt}
)
question = "What are the approaches to Task Decomposition?"
result = qa_chain.invoke({"query": question})
result["result"]

/var/folders/02/m1j4q6jj79zbjw0qtxcldpnc0000gn/T/ipykernel_3407/2784775807.py:14: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  vectorstore = FAISS.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())


'The approaches to Task Decomposition include using LLM with simple prompting, task-specific instructions, and human inputs. Task decomposition involves breaking down large tasks into smaller subgoals for efficient handling of complex tasks and reflecting on past actions for improvement. Challenges in long-term planning and task decomposition include adjusting plans over a lengthy history and effectively exploring the solution space.'

## 95. LCEL (=LangChain Expression Language) - The Runnable Protocol
* https://colab.research.google.com/drive/1iHmhKEhntUy71C_gO4kNqylZQYshAgD1
* https://python.langchain.com/docs/concepts/lcel/

In [6]:
from langchain_core.runnables import RunnableLambda

print(
    type(RunnableLambda(lambda x: x + 1))
)  # <class 'langchain.schema.runnable.RunnableLambda'>

<class 'langchain_core.runnables.base.RunnableLambda'>


In [7]:
chain = RunnableLambda(lambda x: x + 1)

In [8]:
chain.invoke(1), chain.invoke(42)

(2, 43)

In [9]:
# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | (lambda x: x * 2)

print(type(sequence))  # <class 'langchain.schema.runnable.RunnableSequence'>
print("\n\n---")
print(sequence.invoke(1))  # 4
sequence.batch([1, 2, 3])  # [4, 6, 8]

<class 'langchain_core.runnables.base.RunnableSequence'>


---
4


[4, 6, 8]

In [10]:
# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    "mul_2": RunnableLambda(lambda x: x * 2),
    "mul_5": RunnableLambda(lambda x: x * 5),
}
sequence.invoke(1)  # {'mul_2': 4, 'mul_5': 10}

{'mul_2': 4, 'mul_5': 10}

In [11]:
sequence = (
    RunnableLambda(lambda x: x + 1)
    | {
        "mul_2": RunnableLambda(lambda x: x * 2),
        "mul_5": RunnableLambda(lambda x: x * 5),
    }
    | RunnableLambda(lambda x: x["mul_2"] + x["mul_5"])
)
sequence.invoke(1)  # {'mul_2': 4, 'mul_5': 10}

14

In [12]:
from langchain_core.runnables import RunnableParallel

parallel = RunnableParallel(
    {"mul_2": RunnableLambda(lambda x: x * 2), "mul_5": RunnableLambda(lambda x: x * 5)}
)

# This is a dictionary, however it will be composed with other runnables when used in a sequence:
parallel_two = {
    "mul_2": RunnableLambda(lambda x: x["input_one"] * 2),
    "mul_5": RunnableLambda(lambda x: x["input_two"] * 5),
}

print(type(parallel))  # <class 'langchain.schema.runnable.RunnableParallel'>
print(type(parallel_two))  # <class 'dict'>

<class 'langchain_core.runnables.base.RunnableParallel'>
<class 'dict'>


In [13]:
chain = parallel | RunnableLambda(lambda x: x["mul_2"] + x["mul_5"])
chain.invoke(5)

35

In [14]:
second_chain = parallel_two | RunnableLambda(lambda x: x["mul_2"] + x["mul_5"])
second_chain.invoke({"input_one": 5, "input_two": 10})

60

## 96. ChatModels, itemgetter and RAG
* https://colab.research.google.com/drive/1PwupkZARnPadd4i1700WY6Y0u8eiiHxz

In [24]:
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableParallel,
    RunnableLambda,
)
import operator

In [16]:
runnable = RunnableParallel(origin=RunnablePassthrough(), modified=lambda x: x + 1)

print(runnable.invoke(1))  # {'origin': 1, 'modified': 2}


def fake_llm(prompt: str) -> str:  # Fake LLM for the example
    return prompt + " world"


chain = RunnableLambda(fake_llm) | {
    "original": RunnablePassthrough(),  # Original LLM output
    "parsed": lambda text: text[::-1],  # Parsing logic
}

chain.invoke("hello")

{'origin': 1, 'modified': 2}


{'original': 'hello world', 'parsed': 'dlrow olleh'}

In [21]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# chat = ChatOpenAI()
chat = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_FAST)
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

chain = prompt | chat
print(chain)

first=ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}'), additional_kwargs={})]) middle=[] last=ChatGoogleGenerativeAI(model='models/gemini-2.0-flash-lite', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1157a3fe0>, default_metadata=(), model_kwargs={})


In [22]:
print("first", chain.first)
print("last", chain.last)

first input_variables=['topic'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell me a joke about {topic}'), additional_kwargs={})]
last model='models/gemini-2.0-flash-lite' google_api_key=SecretStr('**********') client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1157a3fe0> default_metadata=() model_kwargs={}


In [31]:
# Stream:
print("\n\nStream:\n")
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

# Invoke:
print("\n\nInvoke:\n")
print(chain.invoke({"topic": "bears"}).content)

# Batch:
print("\n\nBatch:\n")
res = chain.batch([{"topic": "bears"}, {"topic": "redhats"}, {"topic": "monks"}])
print(res)
print("\n".join((map(operator.attrgetter("content"), res))))



Stream:

Why don't bears wear shoes?

Because they have bear feet!


Invoke:

Why don't bears wear shoes?

Because they have bear feet!


Batch:

[AIMessage(content="Why don't bears like fast food?\n\nBecause they can't *bear* the wait!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': []}, id='run--99ca5db1-9e50-44f9-b091-9f13d926caee-0', usage_metadata={'input_tokens': 6, 'output_tokens': 22, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}}), AIMessage(content='Why did the Red Hat employee bring a ladder to the Linux conference?\n\nBecause they heard there was a lot of **upgrading** to do!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite', 'safety_ratings': []}, id='run--f6aa99a9-bfe5-42fd-af8c-8a6b93c98269-0', usage_m

### RAG in LCEL

In [32]:
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores.faiss import FAISS

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [38]:
vectorstore = FAISS.from_texts(
    [
        "James Phoenix works as a data engineering and LLM consultant at JustUnderstandingData",
        "James Phoenix has an age of 31 years old.",
    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# model = ChatOpenAI()
model = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_CLEVER)

In [39]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# It's the same as this, but the tuple allows for line breaks:
# {"context": retriever, "question": RunnablePassthrough()} | prompt | model | StrOutputParser()

In [36]:
chain.invoke("What company does James phoenix work at?")

'James Phoenix works at JustUnderstandingData.'

In [40]:
chain.invoke("What is James Phoenix's age?")

'James Phoenix has an age of 31 years old.'

### itemgetter

In [41]:
test = {"data": ["This is a test", "Another entry..."]}

print(itemgetter(test))
print(itemgetter("data")(test))

operator.itemgetter({'data': ['This is a test', 'Another entry...']})
['This is a test', 'Another entry...']


In [43]:
prompt = ChatPromptTemplate.from_template(
    """What is the profession of James Phoenix? His profession is {profession}."""
)

first_chain = RunnableParallel(name=lambda x: "James Phoenix", age=lambda x: 31)

second_chain = {
    # itemgetter is used to get the value from the dictionary from the previous step: (note this is only the previous step, not the whole chain)
    "name": itemgetter("name"),
    "age": itemgetter("age"),
    # You can not use string values, either use itemgetter or a lambda, or RunnablePassthrough
    "profession": lambda x: "Data Engineer",
}

chain = (
    first_chain
    | second_chain
    | prompt
    | ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_FAST)
    | StrOutputParser()
)
chain.invoke({})

"James Phoenix's profession is **Data Engineer**."

## 97. LCEL - Chat Message History and Memory
* https://colab.research.google.com/drive/1dmDHw39-5NNiQtz3s_b8470lXTJYNx8I

In [44]:
from langchain_core.runnables import RunnableMap, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import format_document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts.prompt import PromptTemplate
from operator import itemgetter
from langchain_community.vectorstores.faiss import FAISS
from langchain_openai import OpenAIEmbeddings

### conversational history

In [45]:
vectorstore = FAISS.from_texts(
    [
        "James Phoenix works as a data engineering and LLM consultant at JustUnderstandingData",
        "James is 31 years old.",
    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

In [48]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)
CONDENSE_QUESTION_PROMPT

PromptTemplate(input_variables=['chat_history', 'question'], input_types={}, partial_variables={}, template='Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.\n\nChat History:\n{chat_history}\nFollow Up Input: {question}\nStandalone question:')

In [50]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)
ANSWER_PROMPT

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [52]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs: list,
    document_prompt: PromptTemplate = DEFAULT_DOCUMENT_PROMPT,
    document_separator: str = "\n\n",
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [53]:
from typing import List, Union
from langchain.schema import HumanMessage, SystemMessage, AIMessage


def _format_chat_history(
    chat_history: List[Union[HumanMessage, SystemMessage, AIMessage]]
) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        if isinstance(dialogue_turn, HumanMessage):
            buffer += "\nHuman: " + dialogue_turn.content
        elif isinstance(dialogue_turn, AIMessage):
            buffer += "\nAssistant: " + dialogue_turn.content
        elif isinstance(dialogue_turn, SystemMessage):
            buffer += "\nSystem: " + dialogue_turn.content
    return buffer

In [63]:
chat = ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_CLEVER, temperature=0)
_inputs = RunnableMap(
    standalone_question=RunnablePassthrough.assign(
        chat_history=lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    # | ChatOpenAI(temperature=0)
    | chat
    | StrOutputParser(),
)
_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents,
    "question": lambda x: x["standalone_question"],
}
conversational_qa_chain = _inputs | _context | ANSWER_PROMPT | chat | StrOutputParser()

In [64]:
conversational_qa_chain.invoke(
    {
        "question": "where did James work?",
        "chat_history": [],
    }
)

'James worked at JustUnderstandingData.'

### memory

In [65]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

/var/folders/02/m1j4q6jj79zbjw0qtxcldpnc0000gn/T/ipykernel_6680/3982471492.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [73]:
# First we add a step to load memory
# This adds a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)
# Now we calculate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    # | ChatOpenAI(temperature=0)
    | chat
    | StrOutputParser(),
}
# Now we retrieve the documents

# This is REALLY IMPORTANT as the chain above becomes StrOutputParser() so it will only have one key, which gets passed to the retriever!
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# And finally, we do the part that returns the answers
answer = {
    "answer": (
        final_inputs
        | ANSWER_PROMPT
        # | ChatOpenAI()
        | chat
    ),
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = (
    loaded_memory
    | standalone_question
    | retrieved_documents
    | answer
    # | StrOutputParser()
)

In [74]:
inputs = {"question": "where did James Phoenix work?"}
result = final_chain.invoke(inputs)
print(result)

{'answer': AIMessage(content='James Phoenix worked at JustUnderstandingData.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--452e3306-6fc4-47e5-8a5b-a3593bc25256-0', usage_metadata={'input_tokens': 45, 'output_tokens': 9, 'total_tokens': 54, 'input_token_details': {'cache_read': 0}}), 'docs': [Document(id='fb603293-81a3-4749-ba96-d0acde1c2e77', metadata={}, page_content='James Phoenix works as a data engineering and LLM consultant at JustUnderstandingData'), Document(id='e743fca8-1eb6-4d42-b861-c8bd21bb34bb', metadata={}, page_content='James is 31 years old.')]}


In [75]:
memory.save_context(inputs, {"answer": result["answer"].content})

In [76]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='where did James Phoenix work?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='James Phoenix worked at JustUnderstandingData.', additional_kwargs={}, response_metadata={})]}

## 98. LCEL - Multiple Chains
* https://colab.research.google.com/drive/1W6T-iQ835IEV4fPPiYykY29yCYL8ZIwa

In [85]:
prompt1 = ChatPromptTemplate.from_template("What city was {person} born in?")
prompt2 = ChatPromptTemplate.from_template(
    "What country is the city {city} in? Respond in {language}"
)

# model = ChatOpenAI()
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro")

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

In [89]:
(
    prompt1 | ChatGoogleGenerativeAI(model=GEMINI_MODEL_NAME_FAST) | StrOutputParser()
).invoke({"person": "Barack Obama"})

'Barack Obama is from Honolulu, Hawaii.'

In [87]:
(prompt1 | model | StrOutputParser()).invoke({"person": "Barack Obama"})

"That's a great question with a few parts to it!\n\nWhile he was born in **Honolulu, Hawaii**, the city most closely associated with Barack Obama's adult life and political career is **Chicago, Illinois**.\n\nHere's a quick breakdown:\n\n*   **Birthplace:** Honolulu, Hawaii\n*   **Hometown & Political Base:** Chicago, Illinois. This is where he worked as a community organizer, taught law at the University of Chicago, served as an Illinois State Senator, and launched his campaign for U.S. Senate and President. It's the city he and Michelle Obama call their hometown."

In [88]:
chain2.invoke({"person": "Barack Obama", "language": "Spanish"})

'Ambas ciudades, tanto Chicago como Honolulu, están en los **Estados Unidos**.'

## 99. LCEL - Conditional Logic, Branching and Merging
* https://colab.research.google.com/drive/1f4rSRMDzzAGlSh2zR0oJGIqW2s8NfJeb

In [90]:
from operator import itemgetter
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch, RunnablePassthrough

In [92]:
branch = RunnableBranch(
    (lambda x: x == "hello", lambda x: x),
    (lambda x: isinstance(x, str), lambda x: x.upper()),
    (lambda x: "This is the default case, in case no above lambda functions match."),
)

print(branch.invoke("hello"))  # "hello"
print(branch.invoke("hell"))
print(branch.invoke(None))  # "This is the default case"

hello
HELL
This is the default case, in case no above lambda functions match.


In [93]:
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | ChatOpenAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | ChatOpenAI()
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | ChatOpenAI()
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [94]:
chain.invoke({"input": "scrum"})

"In conclusion, while Scrum offers numerous advantages such as promoting collaboration, enhancing adaptability, and improving efficiency, there are also potential drawbacks that should be considered. These include issues like lack of predictability, high overhead, dependency on team collaboration, limited scalability, focus on short-term goals, and potential for burnout.\n\nTo address these challenges, it's important for teams adopting Scrum to establish clear communication channels, prioritize team well-being, and ensure alignment with stakeholders on expectations. By proactively managing these potential drawbacks, teams can maximize the benefits of the Scrum framework and achieve successful project outcomes.\n\nUltimately, like any project management approach, the effectiveness of Scrum depends on proper implementation, team dynamics, and adaptation to the specific needs of the project. With careful planning and ongoing refinement, teams can leverage the strengths of Scrum while miti

In [108]:
import logging
import langchain
from contextlib import contextmanager
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import BaseCallbackHandler
from typing import Any, Dict

# 1. Configure logging to save output to a file
# logging.basicConfig(
#     filename="langchain_debug.log",
#     filemode="w",
#     level=logging.INFO,
#     format="%(asctime)s - %(levelname)s - %(message)s",
# )
logger = logging.getLogger("langchain")
logger.setLevel(logging.INFO)  # Set the desired logging level (e.g., INFO, DEBUG)

# 2. Create a handler to write to a file
#    'w' mode overwrites the file each time, use 'a' to append
handler = logging.FileHandler("langchain.log", mode="w")

# 3. Create a formatter to define the log message's structure
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)

# 4. Add the handler to the logger
logger.addHandler(handler)

# 5. Prevent logs from propagating to the root logger to avoid duplicates
logger.propagate = False


class MyLoggingCallbackHandler(BaseCallbackHandler):
    """A callback handler that logs events to a given logger."""

    def __init__(self, logger: logging.Logger):
        self.logger = logger

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> None:
        """Log the start of a chain run."""
        self.logger.info(f"Chain started with inputs: {inputs}")

    def on_llm_end(self, response, **kwargs: Any) -> None:
        """Log the end of an LLM call."""
        # The actual response object structure may vary by model provider
        self.logger.info(
            f"LLM generated response: {response.generations[0][0].text[:80]}..."
        )


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")
chain = (
    ChatPromptTemplate.from_template("Tell me a joke about {topic}")
    | llm
    | StrOutputParser()
)

# Instantiate your handler with your logger
my_handler = MyLoggingCallbackHandler(logger=logger)

print("--- Running chain with a custom callback handler ---")
response = chain.invoke(
    {"topic": "cats"}, config={"callbacks": [my_handler]}  # Pass the handler here
)
print("\n--- Final Response ---")
print(response)

--- Running chain with a custom callback handler ---

--- Final Response ---
Why don't cats play poker in the jungle? 

Too many cheetahs!


In [103]:
langchain.debug

False

In [100]:
import langchain

langchain.debug = False

# from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.stdout import StdOutCallbackHandler

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

joke_chain = (
    ChatPromptTemplate.from_template("Tell me a joke about {topic}")
    | llm
    | StrOutputParser()
    | {"joke": RunnablePassthrough(), "topic": RunnablePassthrough()}
)

explain_joke = (
    ChatPromptTemplate.from_template("Explain the joke: {joke}")
    | llm
    | StrOutputParser()
)

benefits_of_joke = (
    ChatPromptTemplate.from_template("List the benefits of this joke: {joke}")
    | llm
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are responsible for generating a small analysis of a joke. The topic will be: {topic}",
            ),
            ("ai", "{joke}. The benefits of this joke are: {benefits}"),
            ("human", "The explanation of the joke is: {explanation}"),
            ("human", "Generate a small analysis of the joke. Analysis: "),
        ]
    )
    | llm
    | StrOutputParser()
)

final_chain = (
    {"topic": RunnablePassthrough()}
    | joke_chain
    | {
        "explanation": explain_joke,
        "benefits": benefits_of_joke,
        "joke": itemgetter("joke"),
        "topic": itemgetter("topic"),
    }
    | final_responder
)

final_chain.invoke(
    {"topic": "bears"},
    # config={"callbacks": [StdOutCallbackHandler()]},
)

'Analysis: This joke hinges entirely on a pun, a form of wordplay that exploits the multiple meanings of a word or the similarity in sound between different words. The effectiveness of the joke lies in the unexpected twist of substituting "bear" (referring to the animal) for "bare" (meaning uncovered). This creates a humorous situation because it takes a literal interpretation of "bear feet" while simultaneously providing a nonsensical, yet understandable, "reason" for the absence of shoes. The humor arises from the incongruity between the expected answer (a logical reason) and the pun-based answer. It\'s a simple, clever joke that leverages the listener\'s knowledge of language and common understanding of bears to achieve its comedic effect.'